In [ ]:
import os
import requests
import json
import sqlite3
import base64

In [ ]:
# connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('image_metadata.db')
c = conn.cursor()

In [ ]:
# create table (if it doesn't exist already)
c.execute('''
    CREATE TABLE IF NOT EXISTS images (
        file_name TEXT,
        file_path TEXT,
        caption TEXT,
        tags TEXT
    )
''')

In [ ]:
# Call the API and get the generated text
api_endpoint = 'http://148.113.143.16:8080'

# prepare the headers for the API request
headers = {
    'Content-Type': 'application/json',
}

In [ ]:
def process_images_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            file_path = os.path.join(folder_path, file_name)

            # check if the image is already in the database
            c.execute('''
                SELECT * FROM images WHERE file_name = ? AND file_path = ?
            ''', (file_name, file_path))
            result = c.fetchone()
            if result is not None:
                print(f"Skipping {file_name} - already in database")
                continue

            # Open the image file in binary mode, encode it in base64, and decode it to ASCII
            with open(file_path, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('ascii')

            # prepare the payload for the API request
            payload = {
                "image": encoded_string
            }

            # send the POST request to the API endpoint
            response = requests.post(api_endpoint, headers=headers, data=json.dumps(payload), timeout=30)
            caption = response.json()['caption']
            tags = response.json()['tags']

            # insert the image metadata into the database
            c.execute('''
                INSERT INTO images VALUES (?, ?, ?, ?)
            ''', (file_name, file_path, caption, tags))

    # commit the changes and close the connection
    conn.commit()
    conn.close()

In [ ]:
folder_path = r"D:\AIML\gimmick\Scene-Sense\test10"  # Update with the actual folder path

# Call the function to process the images
process_images_in_folder(folder_path)

----

In [ ]:
def generate_captions_and_tags(images):
    # API endpoint for the API request
    api_endpoint = 'http://148.113.143.16:9000'

    # Prepare the headers for the API request
    headers = {
        'Content-Type': 'application/json',
    }
    
    for image in images:
        # Open the image file in binary mode, encode it in base64, and decode it to ASCII
        with open(image, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('ascii')

        # Prepare the payload for the API request
        payload = {
            "image": encoded_string
        }

        # Send the POST request to the API endpoint
        response = requests.post(api_endpoint, headers=headers, data=json.dumps(payload), timeout=30)
        
        # Extract the caption and tags from the response
        caption = response.json()['caption']
        tags = response.json()['tags']

    return caption, tags

In [ ]:
caption, tags = generate_captions_and_tags([r"D:\AIML\gimmick\Scene-Sense\test100\laion2b_en_part_00000_000007.png"])
print("Caption: ", caption)
print("Tags: ", tags)

In [ ]:
def preprocess(caption, tags):
    words = caption.split()
    new_tags = tags.split(" | ")
    new_words = []
    for word in words:
        if word in new_tags:
            new_words.append(word)
    for tags in new_tags:
        if tags not in new_words:
            new_words.append(tags)
    return " ".join(new_words)

better_tags = preprocess(caption, tags)

In [ ]:
print("Caption: ", caption)
print("Better Tags: ", better_tags)

In [ ]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('image_metadata.db')
c = conn.cursor()

In [ ]:
# Create table (if it doesn't exist already)
c.execute('''
    CREATE TABLE IF NOT EXISTS images (
        file_name TEXT,
        file_path TEXT,
        caption TEXT,
        tags TEXT
    )
''')

# Insert the image metadata into the database
c.execute('''
    INSERT INTO images VALUES (?, ?, ?, ?)
''', (file_name, file_path, caption, ', '.join(tags)))

# Commit the changes and close the connection
conn.commit()
conn.close()